In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
dataset = '/content/drive/MyDrive/infrance training/model/point_history_classifier/point_history.csv'
model_save_path = '/content/drive/MyDrive/infrance training/model/point_history_classifier/weights.hdf5'
tflite_save_path = '/content/drive/MyDrive/infrance training/model/point_history_classifier/model.tflite'

In [ ]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (16 * 2) + 1)))

In [ ]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(16 * 2, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                250       
                                                                 
 dense_2 (Dense)             (None, 4)                 44        
                                                                 
Total params: 1,086
Trainable params: 1,086
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
17/32 [==============>...............] - ETA: 0s - loss: 1.3969 - accuracy: 0.3543  
Epoch 1: saving model to /content/drive/MyDrive/infrance training/model/point_history_classifier/weights.hdf5
32/32 [==============================] - 4s 22ms/step - loss: 1.3918 - accuracy: 0.4013 - val_loss: 1.3753 - val_accuracy: 0.4924
Epoch 2/1000
17/32 [==============>...............] - ETA: 0s - loss: 1.3778 - accuracy: 0.4646
Epoch 2: saving model to /content/drive/MyDrive/infrance training/model/point_history_classifier/weights.hdf5
32/32 [==============================] - 0s 6ms/step - loss: 1.3767 - accuracy: 0.4554 - val_loss: 1.3624 - val_accuracy: 0.4826
Epoch 3/1000
32/32 [==============================] - ETA: 0s - loss: 1.3628 - accuracy: 0.4642
Epoch 3: saving model to /content/drive/MyDrive/infrance training/model/point_history_classifier/weights.hdf5
32/32 [==============================] - 0s 6ms/step - loss: 1.3628 - accuracy: 0.4642 - val_loss: 1.3472 - val_accuracy:

In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[0.91722494 0.01737202 0.02012045 0.04528268]
0


In [ ]:
model.save(model_save_path, include_optimizer=False)
model = tf.keras.models.load_model(model_save_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /tmp/tmp2nq5thub/assets


6304